In [ ]:
# License: BSD
# Adapted from Sasank Chilamkurthy pytorch tutorial for image classification.
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../StateFarmDistractedDriverDetection/dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train','val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=4)
              for x in ['train','val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
print(dataset_sizes)
class_names = image_datasets['train'].classes

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
# Define the neural network used
#model_ft = models.alexnet(pretrained =True)
model_ft = models.resnet18(pretrained= True)
#model_ft = models.vgg16(pretrained = True)

In [ ]:
print (model_ft.parameters)

## Use the following for AlexNet
net = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
        )
model_ft.classifier = net

## Use the following for VGG-16
net = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
        )
model_ft.classifier = net

In [ ]:
# Freeze all layers except for the final fully connected layer.
for param in model_ft.parameters():
    param.requires_grad = False

## Use the following for AlexNet
### Replace the final layer to classify into one of 10 classes.
net2 = nn.Linear(4096, 10)
model_ft.classifier.add_module('classify',net2)

In [ ]:
## Use the following for Resnet18
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10)

## Use this for VGG-16 
### Replace the final layer to classify into one of 10 classes.
net2 = nn.Linear(4096, 10)
model_ft.classifier.add_module('classify',net2)


In [ ]:
if torch.cuda.is_available():
    model_ft = model_ft.cuda()
    print ("Transfer model to GPU")

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# Use this for AlexNet
#optimizer = optim.SGD(model_ft.classifier.classify.parameters(),lr=0.001, momentum=0.9)

# Use this for Resnet18
optimizer = optim.SGD(model_ft.fc.parameters(),lr=0.001, momentum=0.9)

# Use this for VGG16
#optimizer = optim.SGD(model_ft.classifier.classify.parameters(),lr=0.001, momentum=0.9)

# Can add other args such as weight decay etc.

In [ ]:
# use Adam
# learning_rate = 0.001
# optimizer = optim.Adam(model_ft.fc.parameters(),lr=learning_rate)

In [ ]:
import math

In [ ]:
batch_size = 64
NB = math.ceil(len(image_datasets['train']) /batch_size) 
NB = int(NB)
print (NB)

In [ ]:
import torch.autograd as ag

In [ ]:
import time

In [ ]:
num_epochs = 50
accu_test = 0.0
train_accu = 0.0
Time=[]
testaccu = []
trainloss = []

In [ ]:
for epoch in range(num_epochs):
    train_loss = 0.0
    running_loss = 0.0
    train_accu = 0.0
    correct = 0
    # Set model to training mode
    model_ft.train()
    t0 = time.time()   
    for i,(inputs,classes) in enumerate(dataloaders['train'],0):
        
        # Obtain a batch of training data
        inputs = inputs.cuda()
        classes = classes.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward
        inputs = ag.Variable(inputs,requires_grad=True)
        classes = ag.Variable(classes, requires_grad =False)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        # Error evaluation
        loss = criterion(outputs,classes)
        
        # Back Propagation
        loss.backward()
        
        # Parameter update
        optimizer.step()
        
        # Print training loss per epoch
        running_loss += loss[0]
        if i%313 == 312:
            print ('[%d] train loss: %.3f'% (epoch+1, np.mean(running_loss.cpu().data.numpy())))
            trainloss.append(np.mean(running_loss.cpu().data.numpy()))
        running_loss = 0.0
    
    
# Run trained model on test data
    for i,(inputs,classes) in enumerate(dataloaders['val'],0):
        inputs = inputs.cuda()
        classes = classes.cuda()
        inputs = ag.Variable(inputs,requires_grad=True)
        classes = ag.Variable(classes, requires_grad =False)
        outputs = model_ft(inputs)
    
    # Calculate Accuracy
        outputs_np = outputs.cpu().data.numpy().T.argmax(axis=0)
        classes_np = classes.cpu().data.numpy()
    
        correct += np.mean(np.equal(classes_np,outputs_np))
    testaccu.append(correct/len(dataloaders['val'])*100)
    #     print accuracy per epoch
    print ('[%d] testaccu %.3f'% (epoch+1, correct/len(dataloaders['val'])*100))
    print('{} seconds'.format(time.time() - t0))
    Time.append(time.time() - t0)
print ("Finished Training")
    

In [ ]:
# save results for plotting
data_array_1 = np.array(Time)
data_array_2 = np.array(testaccu)
data_array_3 = np.array(trainloss)
# saving...
np.savetxt('time_ResNet.csv',data_array_1,delimiter=',')
np.savetxt('testaccu_ResNet.csv',data_array_2,delimiter=',')
np.savetxt('trainloss_ResNet.csv',data_array_3,delimiter=',')
print ('Finish saving csv file')

In [ ]:
# AlexNet accuracy is 73.10385338345865591
# Resnet accuracy is 94.54534774436091027
# Vgg16 accuracy is 81.29934210526316463